In [ ]:
import numpy as np
import time
import tensorflow
from tensorflow.keras.applications.efficientnet import EfficientNetB3, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import GlobalAveragePooling2D
tensorflow.__version__

In [21]:
base_model = EfficientNetB3(include_top=False,
                            weights='imagenet',
                            input_tensor=Input(shape=(300, 300, 3)))
x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

In [1]:
# model save using keras; IMPORTANT to specify version number (in this case time.time())
model_save_path = './img_cls/{}'.format(int(time.time()))
print(model_save_path)
model.save(model_save_path, save_format='tf')

In [28]:
# alternate way of saving using tf saved_model method
# model_save_path = './img_cls/{}'.format(int(time.time()))
# print(model_save_path)
# tensorflow.saved_model.save(model, model_save_path)

In [4]:
'''
# This is to test if your model works with vanila docker
# absolute path is a must while binding volume

docker run -p 8501:8501 --name tfs1 \
--mount type=bind,source=/home/ubuntu/Documents/jbs/examples/misc/embeddings_server/img_cls/,target=/models/img_cls \
-e MODEL_NAME=img_cls -t tensorflow/serving


Usefull commands
docker restart tfs1
docker stop tfs1
docker start tfs1
docker rm tfs1
'''

In [26]:
!docker restart tfs1

tfs1


In [ ]:
# test api

In [9]:
import matplotlib.pyplot as plt
import requests
import base64
import json
import numpy as np

In [29]:
img = np.random.random((1, 30,30,3))
img.shape

(1, 30, 30, 3)

In [30]:
%%time
preds = model.predict(img)

CPU times: user 2.41 s, sys: 120 ms, total: 2.53 s
Wall time: 4.37 s


In [31]:
preds.shape

(1, 1536)

In [32]:
%%time


CPU times: user 608 µs, sys: 11 µs, total: 619 µs
Wall time: 628 µs


In [56]:
%%time
imlst = img.tolist()
url = 'http://localhost:8501/v1/models/img_cls:predict'
data = json.dumps({"signature_name": "serving_default", "instances":imlst})
headers = {"content-type": "application/json"}
json_response = requests.post(url, data=data, headers=headers)
out = np.array((json.loads(json_response.text)['predictions']))
out

CPU times: user 4.61 ms, sys: 3.9 ms, total: 8.52 ms
Wall time: 18.3 ms


array([[-0.08056594, -0.1896586 , -0.07315277, ..., -0.2767792 ,
        -0.22485656,  0.1715697 ]])

In [57]:
# test

In [45]:
img = np.random.random((1, 30,30,3))
imlst = img.tolist()
url = 'http://localhost:8501/v1/models/img_cls:predict'

In [51]:
%%time
for _ in range(2):
    json_response = requests.post(url, data=data, headers=headers)
    out = np.array((json.loads(json_response.text)['predictions']))
    print(out)

[[-0.08012704 -0.19066724 -0.07158289 ... -0.27662969 -0.2239328
   0.16618751]]
[[-0.08012704 -0.19066724 -0.07158289 ... -0.27662969 -0.2239328
   0.16618751]]
CPU times: user 5.71 ms, sys: 4.03 ms, total: 9.74 ms
Wall time: 30 ms


In [36]:
# docker compose

In [34]:
%%time
!curl localhost:5001/predict

curl: (7) Failed to connect to localhost port 5001: Connection refused
CPU times: user 3.5 ms, sys: 28.1 ms, total: 31.6 ms
Wall time: 149 ms
